## Kütüphanelerin yüklenmesi

In [34]:
!pip install pandas transformers sentence-transformers scikit-learn matplotlib seaborn faiss-cpu

In [35]:
import random
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import faiss

## Veri setini yükleme

In [36]:
data = pd.read_csv("gsm8k_tr.csv")


## Rasgele 1000 kaydın alınması

In [37]:
total_data_length = data.__len__()

random.seed(111)
indices = random.sample(range(total_data_length), 1000)

sampled_data = data.iloc[indices].reset_index(drop=True)
data_frame = pd.DataFrame(sampled_data)

## Modellerin yüklenmesi

In [38]:
model_names = [
    "sentence-transformers/LaBSE",
]

models = [SentenceTransformer(name) for name in model_names]

# Tokenizer ve model yükleme fonksiyonu
def load_model(name):
    tokenizer = AutoTokenizer.from_pretrained(name)
    model = AutoModel.from_pretrained(name)
    return tokenizer, model

# Temsilleri hesaplama fonksiyonu
def get_embeddings(model, texts):
    return model.encode(texts, convert_to_tensor=True)

embeddings = {}
for name, model in zip(model_names, models):
    embeddings[name] = {
        'questions': get_embeddings(model, data_frame['question']),
        'answers': get_embeddings(model, data_frame['answer'])
    }


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

## Benzerlik

In [39]:
def calculate_accuracy(embeddings, text_type='questions'):
    accuracies = {}
    for name, embed in embeddings.items():
        if text_type == 'questions':
            source_embeddings = embed['questions']
            target_embeddings = embed['answers']
        else:
            source_embeddings = embed['answers']
            target_embeddings = embed['questions']
        
        index = faiss.IndexFlatIP(source_embeddings.shape[1])
        faiss.normalize_L2(source_embeddings.numpy())
        faiss.normalize_L2(target_embeddings.numpy())
        index.add(target_embeddings.numpy())
        
        D, I = index.search(source_embeddings.numpy(), 5)
        
        top1 = sum([1 if i[0] == j else 0 for i, j in zip(I, range(len(I)))]) / len(I)
        top5 = sum([1 if j in i else 0 for i, j in zip(I, range(len(I)))]) / len(I)
        
        accuracies[name] = {'top1': top1, 'top5': top5}
    
    return accuracies

question_accuracies = calculate_accuracy(embeddings, text_type='questions')
answer_accuracies = calculate_accuracy(embeddings, text_type='answers')

print("Question Accuracies:", question_accuracies)
print("Answer Accuracies:", answer_accuracies)


TypeError: can't convert mps:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.